# Getting the respective templates in an excel

In [14]:
import pandas as pd
import docx
import os
from openai import OpenAI
import json
import re
import dotenv

dotenv.load_dotenv()
openai_key = os.getenv('OPENAI_KEY')

In [19]:
class SportsTemplateExtractor:
    def __init__(self, openai_api_key, docx_folder=".", output_file="sports_templates.xlsx"):
        self.client = OpenAI(api_key=openai_api_key)
        self.docx_folder = docx_folder
        self.output_file = output_file
        self.templates_df = pd.DataFrame(columns=[
            'sport', 'event_category', 'gender', 'event_name', 
            'event_type', 'template', 'fields', 'sample_data'
        ])
    
    def read_docx(self, file_path):
        """Extract text from docx file"""
        try:
            doc = docx.Document(file_path)
            full_text = []
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    full_text.append(paragraph.text)
            return '\n'.join(full_text)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return ""
    
    def extract_sport_name(self, filename):
        """Extract sport name from filename"""
        sport_name = filename.replace('2023 ', '').replace(' Results', '')
        sport_name = sport_name.replace(' Whatsapp Template.docx', '')
        sport_name = sport_name.replace(' WhatsApp Template.docx', '')
        sport_name = sport_name.replace('...', ' ')
        sport_name = sport_name.replace(' - ', ' ')
        
        # Handle special cases
        special_cases = {
            'Volleyball (Indoor)': 'Volleyball Indoor',
            'Volleyball (Beach)': 'Volleyball Beach',
            'Hockey (Indoor)': 'Hockey Indoor',
            'Hockey (Field)': 'Hockey Field',
            'Traditional Boat Ra': 'Traditional Boat Racing',
            'Teqball (Demo Spo': 'Teqball',
            'Table Tennis - Res': 'Table Tennis',
            'Sepak Takraw Res': 'Sepak Takraw',
            'Pencak Silat Result': 'Pencak Silat',
            'Finswimming - Res': 'Finswimming',
            'Gymnastics - Resu': 'Gymnastics'
        }
        
        for key, value in special_cases.items():
            if key in filename:
                return value
        
        return sport_name.strip()
    
    def process_with_chatgpt(self, text, sport_name):
        """Use ChatGPT to extract templates from text"""
        prompt = f"""
        Analyze this sports results template for {sport_name} and extract all unique templates.
        
        Text: {text}
        
        For each unique template pattern found, identify:
        1. Event category (e.g., "WOMEN 200M HEAT", "MEN POLE VAULT FINAL")
        2. Gender (Men/Women/Mixed)
        3. Event name (e.g., "200M", "Pole Vault", "Marathon")
        4. Event type (Heat/Final/Qualification/Preliminary/etc.)
        5. Template structure with variables in {{}} format
        6. List of all variable fields
        7. Sample data from the text
        
        Look for patterns like:
        - Event headers with asterisks: *SPORT – GENDER EVENT TYPE*
        - Athlete names with country codes: NAME (SGP)
        - Performance metrics: Time:, Height:, Score:, etc.
        - Placement information: finished X out of Y
        - Additional comments about records, advancement, etc.
        
        Return as JSON with this structure:
        {{
            "templates": [
                {{
                    "event_category": "WOMEN 200M HEAT",
                    "gender": "Women",
                    "event_name": "200M",
                    "event_type": "Heat",
                    "template": "*{{SPORT}} – {{GENDER}} {{EVENT}} {{TYPE}}*\\n{{NAME}} (SGP)\\nTime: {{TIME}}. {{PRONOUN}} finished {{PLACEMENT}} out of {{TOTAL}} and {{ADVANCEMENT_STATUS}}. {{ADDITIONAL_COMMENTS}}",
                    "fields": ["SPORT", "GENDER", "EVENT", "TYPE", "NAME", "TIME", "PRONOUN", "PLACEMENT", "TOTAL", "ADVANCEMENT_STATUS", "ADDITIONAL_COMMENTS"],
                    "sample_data": "Actual example from the text"
                }}
            ]
        }}
        
        Extract ALL unique template patterns from the text.
        """
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are an expert at analyzing sports result templates and extracting structured data. Always return valid JSON."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=4000
            )
            
            content = response.choices[0].message.content
            if content.startswith('```json'):
                content = content.replace('```json', '').replace('```', '')
            
            result = json.loads(content)
            return result
        except json.JSONDecodeError as e:
            print(f"JSON decode error for {sport_name}: {e}")
            return {"templates": []}
        except Exception as e:
            print(f"Error processing {sport_name} with ChatGPT: {e}")
            return {"templates": []}
    
    def process_single_file(self, file_path):
        """Process a single docx file"""
        filename = os.path.basename(file_path)
        sport_name = self.extract_sport_name(filename)
        
        print(f"Processing: {sport_name}")
        
        text = self.read_docx(file_path)
        if not text:
            print(f"No text found in {filename}")
            return
        
        result = self.process_with_chatgpt(text, sport_name)
        
        templates_added = 0
        for template_data in result.get('templates', []):
            new_row = {
                'sport': sport_name,
                'event_category': template_data.get('event_category', ''),
                'gender': template_data.get('gender', ''),
                'event_name': template_data.get('event_name', ''),
                'event_type': template_data.get('event_type', ''),
                'template': template_data.get('template', ''),
                'fields': json.dumps(template_data.get('fields', [])),
                'sample_data': template_data.get('sample_data', '')
            }
            self.templates_df = pd.concat([self.templates_df, pd.DataFrame([new_row])], ignore_index=True)
            templates_added += 1
        
        print(f"  Added {templates_added} templates for {sport_name}")
    
    def process_all_files(self):
        """Process all docx files in folder"""
        docx_files = [f for f in os.listdir(self.docx_folder) if f.endswith('.docx')]
        print(f"Found {len(docx_files)} DOCX files to process")
        
        for i, filename in enumerate(docx_files, 1):
            print(f"\n[{i}/{len(docx_files)}] Processing: {filename}")
            file_path = os.path.join(self.docx_folder, filename)
            self.process_single_file(file_path)
    
    def save_to_excel(self):
        """Save templates to Excel"""
        if len(self.templates_df) == 0:
            print("No templates found to save!")
            return
            
        with pd.ExcelWriter(self.output_file, engine='openpyxl') as writer:
            self.templates_df.to_excel(writer, sheet_name='All Templates', index=False)
            
            summary_df = self.templates_df.groupby('sport').size().reset_index(name='template_count')
            summary_df.to_excel(writer, sheet_name='Summary by Sport', index=False)
            
            for sport in self.templates_df['sport'].unique():
                sport_df = self.templates_df[self.templates_df['sport'] == sport]
                sheet_name = sport[:31]
                sport_df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"\nTemplates saved to: {self.output_file}")
        print(f"Total templates extracted: {len(self.templates_df)}")
        print(f"Sports covered: {len(self.templates_df['sport'].unique())}")
    
    def run(self):
        """Main execution method"""
        print("Starting Sports Template Extraction...")
        self.process_all_files()
        self.save_to_excel()
        print("Extraction complete!")



OPENAI_API_KEY = openai_key 
DOCX_FOLDER = "Major-Games-Reporting/Whatsapp Templates/Whatsapp Templates" 
OUTPUT_FILE = "sports_templates.xlsx"


# Run extraction
extractor = SportsTemplateExtractor(
    openai_api_key=OPENAI_API_KEY,
    docx_folder=DOCX_FOLDER,
    output_file=OUTPUT_FILE
)

extractor.run()


Starting Sports Template Extraction...
Found 37 DOCX files to process

[1/37] Processing: 2023 Gymnastics - Results Whatsapp Template.docx
Processing: Gymnastics
  Added 3 templates for Gymnastics

[2/37] Processing: 2023 Sepak Takraw Results Whatsapp Template.docx
Processing: Sepak Takraw
  Added 6 templates for Sepak Takraw

[3/37] Processing: 2023 E-Sports Results Whatsapp Template.docx
Processing: E-Sports
  Added 5 templates for E-Sports

[4/37] Processing: 2023 Billiards Sports Results Whatsapp Template.docx
Processing: Billiards Sports
  Added 5 templates for Billiards Sports

[5/37] Processing: 2023 Volleyball (Beach) Results WhatsApp Template.docx
Processing: Volleyball Beach
  Added 6 templates for Volleyball Beach

[6/37] Processing: 2023 Traditional Boat Race Results WhatsApp Template.docx
Processing: Traditional Boat Racing
  Added 7 templates for Traditional Boat Racing

[7/37] Processing: 2023 Floorball - Results WhatsApp Template.docx
Processing: Floorball -
  Added 4 t

# Mapping Sports to Templates

In [5]:
import pandas as pd
import re
from typing import Dict, List, Tuple, Optional
import ast

class SportsTemplateMapper:
    def __init__(self, file_path: str):
        # Load the data
        self.all_sports = pd.read_excel(file_path, sheet_name='all_sports_raw')
        self.template_raw = pd.read_excel(file_path, sheet_name='template_raw')
        self.template_type = pd.read_excel(file_path, sheet_name='template_type')
        self.sport_result_mapping = pd.read_excel(file_path, sheet_name='sport_result_mapping')
        
        # Clean the template_type data and add IDs
        self._prepare_template_type()
        
        # Initialize normalization mappings
        self.sport_map = self._build_sport_map()
        self.round_map = self._build_round_map()
        
    def _prepare_template_type(self):
        """Clean template_type data and add proper IDs"""
        # Remove rows where template is NaN
        self.template_type = self.template_type.dropna(subset=['template'])
        
        # Add proper IDs if missing
        if 'id' not in self.template_type.columns or self.template_type['id'].isna().all():
            self.template_type['id'] = [f'TYPE_{i+1:03d}' for i in range(len(self.template_type))]
        else:
            # Fill missing IDs
            for idx, row in self.template_type.iterrows():
                if pd.isna(row.get('id')):
                    self.template_type.at[idx, 'id'] = f'TYPE_{idx+1:03d}'
        
        # Reset index
        self.template_type = self.template_type.reset_index(drop=True)
        
    def _build_sport_map(self) -> Dict[str, str]:
        """Normalize sport names to standard format"""
        sport_map = {}
        
        # Get unique sports from template_raw, all_sports, and sport_result_mapping
        template_sports = set(self.template_raw['sport'].dropna().unique())
        all_sports_sports = set(self.all_sports['Sports'].dropna().unique())
        result_mapping_sports = set(self.sport_result_mapping['sport'].dropna().unique())
        unique_sports = template_sports | all_sports_sports | result_mapping_sports
        
        for sport in unique_sports:
            normalized = sport.upper().strip()
            
            # Handle special cases based on your data
            if 'AQUATICS' in normalized:
                if 'SWIMMING' in normalized:
                    normalized = 'SWIMMING'
                elif 'DIVING' in normalized:
                    normalized = 'DIVING'
                elif 'WATERPOLO' in normalized or 'WATER POLO' in normalized:
                    normalized = 'WATER POLO'
            elif 'VOLLEYBALL' in normalized:
                if 'BEACH' in normalized:
                    normalized = 'VOLLEYBALL BEACH'
                elif 'INDOOR' in normalized:
                    normalized = 'VOLLEYBALL INDOOR'
                else:
                    normalized = 'VOLLEYBALL'
            elif 'HOCKEY' in normalized:
                if 'FIELD' in normalized:
                    normalized = 'HOCKEY FIELD'
                elif 'INDOOR' in normalized:
                    normalized = 'HOCKEY INDOOR'
                elif 'ICE' in normalized:
                    normalized = 'ICE HOCKEY'
                else:
                    normalized = 'HOCKEY'
            elif 'BILLIARDS' in normalized or 'CUESPORTS' in normalized:
                normalized = 'CUESPORTS'
            elif 'TRADITIONAL BOAT' in normalized or 'DRAGON BOAT' in normalized:
                normalized = 'TRADITIONAL BOAT RACE'
            elif 'E-SPORTS' in normalized or 'ESPORTS' in normalized:
                normalized = 'ESPORTS'
            
            sport_map[sport] = normalized
            
        # Hardcode all missing sports from sport_result_mapping
        missing_sports = {
            'AQUATHLON': 'AQUATHLON',
            'DUATHLON': 'DUATHLON', 
            'TRIATHLON': 'TRIATHLON',
            'ARTISTIC SWIMMING': 'ARTISTIC SWIMMING',
            'ATHLETICS': 'ATHLETICS',
            'BADMINTON': 'BADMINTON',
            'BASEBALL': 'BASEBALL',
            'BASKETBALL': 'BASKETBALL',
            'BOWLING': 'BOWLING',
            'BOXING': 'BOXING',
            'CANOEING': 'CANOEING',
            'CHESS': 'CHESS',
            'CRICKET': 'CRICKET',
            'CUESPORTS': 'CUESPORTS',
            'CYCLING ROAD RACE': 'CYCLING',
            'CYCLING': 'CYCLING',
            'CYCLING MOUNTAIN BIKING': 'CYCLING',
            'DIVING': 'DIVING',
            'DRAGON BOAT': 'TRADITIONAL BOAT RACE',
            'EQUESTRIAN': 'EQUESTRIAN',
            'ESPORTS': 'ESPORTS',
            'FENCING': 'FENCING',
            'FLOORBALL': 'FLOORBALL',
            'FLYING DISC': 'FLYING DISC',
            'FOOTBALL': 'FOOTBALL',
            'GOLF': 'GOLF',
            'GYMNASTICS': 'GYMNASTICS',
            'HANDBALL': 'HANDBALL',
            'HOCKEY': 'HOCKEY',
            'ICE HOCKEY': 'ICE HOCKEY',
            'ICE SKATING': 'ICE SKATING',
            'JET SKI': 'JET SKI',
            'JUDO': 'JUDO',
            'JU-JITSU': 'JU-JITSU',
            'KABADDI': 'KABADDI',
            'KARATE': 'KARATE',
            'KICKBOXING': 'KICKBOXING',
            'MODERN PENTATHLON': 'MODERN PENTATHLON',
            'MUAY THAI': 'MUAY THAI',
            'NETBALL': 'NETBALL',
            'OPEN WATER SWIMMING': 'OPEN WATER SWIMMING',
            'PARAGLIDING': 'PARAGLIDING',
            'PENCAK SILAT': 'PENCAK SILAT',
            'PETANQUE': 'PETANQUE',
            'POLO': 'POLO',
            'ROLLER SPORTS': 'ROLLER SPORTS',
            'ROWING': 'ROWING',
            'RUGBY': 'RUGBY',
            'SAILING': 'SAILING',
            'SEPAKTAKRAW': 'SEPAKTAKRAW',
            'SHOOTING': 'SHOOTING',
            'SOFTBALL': 'SOFTBALL',
            'SPORT CLIMBING': 'SPORT CLIMBING',
            'SQUASH': 'SQUASH',
            'SWIMMING': 'SWIMMING',
            'TABLE TENNIS': 'TABLE TENNIS',
            'TAEKWONDO': 'TAEKWONDO',
            'TENNIS': 'TENNIS',
            'TEQBALL': 'TEQBALL',
            'TUG OF WAR': 'TUG OF WAR',
            'VOLLEYBALL': 'VOLLEYBALL',
            'WATER POLO': 'WATER POLO',
            'WATERSKI & WAKEBOARD': 'WATERSKI & WAKEBOARD',
            'WEIGHTLIFTING': 'WEIGHTLIFTING',
            'WOODBALL': 'WOODBALL',
            'WRESTLING': 'WRESTLING',
            'ARCHERY': 'ARCHERY',
            'FLAG FOOTBALL': 'FLAG FOOTBALL',
            'LACROSSE': 'LACROSSE',
            'SURFING': 'SURFING',
            'WUSHU': 'WUSHU',
            'FINSWIMMING': 'FINSWIMMING'
        }
        
        # Add all missing sports to the sport_map
        for original, normalized in missing_sports.items():
            sport_map[original] = normalized
            
        return sport_map
    
    def _build_round_map(self) -> Dict[str, str]:
        """Normalize round names to standard format"""
        return {
            'Qualification': 'QUALIFICATION',
            'Final': 'FINALS',
            'Team Final': 'FINALS',
            'Round Robin': 'ROUND ROBIN',
            'Team Round Robin': 'ROUND ROBIN',
            '3rd/4th Placing Match': 'BRONZE MEDAL PLAYOFF',
            'Bronze Medal Playoff': 'BRONZE MEDAL PLAYOFF',
            'Semi Finals': 'SEMI FINALS',
            'Semi Final': 'SEMI FINALS',
            'Quarter Finals': 'QUARTER FINALS',
            'Quarter Final': 'QUARTER FINALS',
            'Preliminary Round': 'PRELIMINARY ROUND',
            'Preliminaries': 'PRELIMINARY ROUND',
            'Group Stage': 'GROUP STAGE',
            'Pool Stage': 'POOL STAGE',
            'POOL STAGE': 'POOL STAGE',
            'Pool': 'POOL',
            'Heat': 'HEAT',
            'Heats': 'HEAT',
            'Grand Final': 'GRAND FINAL',
            'Round of 32': 'ROUND OF 32',
            'Round of 16': 'ROUND OF 16',
            'Round of 12': 'ROUND OF 12',
            'Table of 16': 'TABLE OF 16',
            'Elimination Round': 'ELIMINATION ROUND',
            'Upper Bracket': 'UPPER BRACKET',
            'Fleet Racing': 'FLEET RACING DAY',
            'Open Fleet Racing': 'FLEET RACING DAY',
            'Individual Final': 'FINALS',
            'Relay Final': 'FINALS',
            'Finals': 'FINALS',
            'Round': 'ROUND ROBIN',
            'Round One': 'ROUND ONE',
            '7th and 8th Placing': 'PLACING MATCH',
            '5th to 8th Placing': 'PLACING MATCH',
            '5th to 8th Placing Playoffs': 'PLACING MATCH',
            '3RD PLACING MATCH': 'BRONZE MEDAL PLAYOFF',
            'Finals (Gunshu)': 'FINALS',
            'Finals (Taijiquan)': 'FINALS'
        }
    
    def _infer_team_flag(self, event_name: str, template_text: str, fields: List[str]) -> str:
        """Infer if this is a team event"""
        if pd.isna(event_name):
            event_name = ""
        if pd.isna(template_text):
            template_text = ""
            
        team_indicators = [
            'TEAM', 'RELAY', 'DOUBLES', 'PAIRS', 'CREW', 'TOURNAMENT',
            'TEAM_MEMBERS', 'PLAYER_NAMES', 'REGU', 'DUILIAN'
        ]
        
        text_to_check = f"{event_name} {template_text} {' '.join(fields) if fields else ''}".upper()
        
        for indicator in team_indicators:
            if indicator in text_to_check:
                return 'yes'
                
        return 'no'
    
    def _get_result_type(self, sport_norm: str, event_norm: str) -> str:
        """Get result type from sport_result_mapping"""
        # Try exact match first
        match = self.sport_result_mapping[
            (self.sport_result_mapping['sport'].str.upper() == sport_norm) & 
            (self.sport_result_mapping['value'].str.upper() == event_norm.upper())
        ]
        
        if not match.empty:
            return match.iloc[0]['resultType']
        
        # Try partial match on sport
        sport_matches = self.sport_result_mapping[
            self.sport_result_mapping['sport'].str.upper() == sport_norm
        ]
        
        if not sport_matches.empty:
            return sport_matches.iloc[0]['resultType']
        
        # Default fallback based on sport type
        if sport_norm in ['SWIMMING', 'ATHLETICS', 'CYCLING', 'AQUATHLON', 'DUATHLON', 'TRIATHLON']:
            return 'time'
        elif sport_norm in ['DIVING', 'GYMNASTICS', 'FENCING', 'ARTISTIC SWIMMING']:
            return 'score'
        elif sport_norm in ['ATHLETICS'] and any(x in event_norm for x in ['JUMP', 'THROW', 'PUT']):
            return 'height'
        elif sport_norm in ['WEIGHTLIFTING']:
            return 'weight'
        else:
            return 'score'
    
    def _infer_medal_awarded(self, round_norm: str, template_text: str) -> str:
        """Check if medals are awarded in this round"""
        if pd.isna(template_text):
            template_text = ""
            
        medal_rounds = ['FINALS', 'BRONZE MEDAL PLAYOFF', 'GRAND FINAL']
        if round_norm in medal_rounds:
            return 'yes'
            
        # Check template text for medal mentions
        if 'medal' in template_text.lower():
            return 'yes'
            
        return 'no'
    
    def _infer_advancement(self, template_text: str, round_norm: str) -> str:
        """Check if this round has advancement to next stage"""
        if pd.isna(template_text):
            template_text = ""
            
        advancement_indicators = [
            'advance', 'qualified', 'next', 'semi', 'final', 'quarter'
        ]
        
        text_lower = template_text.lower()
        for indicator in advancement_indicators:
            if indicator in text_lower:
                return 'yes'
                
        # Finals typically don't advance
        if round_norm in ['FINALS', 'GRAND FINAL']:
            return 'no'
            
        return 'no'
    
    def derive_template_signature(self, row) -> Dict:
        """Derive signature from template_raw row"""
        sport_norm = self.sport_map.get(row['sport'], row['sport'].upper() if pd.notna(row['sport']) else 'UNKNOWN')
        
        # Normalize event
        event_name = row.get('event_name', '') or ''
        if pd.isna(event_name):
            event_name = ''
        event_norm = event_name.upper().strip()
        
        # Normalize round
        event_type = row.get('event_type', '') or ''
        if pd.isna(event_type):
            event_type = ''
        round_norm = self.round_map.get(event_type, event_type.upper().strip())
        
        # Parse fields
        fields_str = row.get('fields', '[]')
        try:
            if pd.isna(fields_str):
                fields = []
            elif isinstance(fields_str, str):
                fields = ast.literal_eval(fields_str)
            else:
                fields = []
        except:
            fields = []
            
        template_text = row.get('template', '') or ''
        if pd.isna(template_text):
            template_text = ''
        
        # Derive signature components
        team_flag = self._infer_team_flag(event_name, template_text, fields)
        result_type = self._get_result_type(sport_norm, event_norm)
        medal_awarded = self._infer_medal_awarded(round_norm, template_text)
        advancement = self._infer_advancement(template_text, round_norm)
        
        return {
            'sport_norm': sport_norm,
            'event_norm': event_norm,
            'team_flag': team_flag,
            'result_type': result_type,
            'round_norm': round_norm,
            'medal_awarded': medal_awarded,
            'advancement': advancement
        }
    
    def derive_template_type_signature(self, row) -> Dict:
        """Derive signature from template_type row"""
        # Extract from the template_type row
        team_flag = 'yes' if str(row.get('team', '')).lower() == 'yes' else 'no'
        result_type = row.get('resultType', 'score')
        if pd.isna(result_type):
            result_type = 'score'
            
        round_val = row.get('round', '')
        if pd.isna(round_val):
            round_val = ''
        round_norm = str(round_val).upper()
        
        medal_awarded = 'yes' if str(row.get('medalStatus', '')).lower() == 'yes' else 'no'
        advancement = 'yes' if str(row.get('advancementStatus', '')).lower() == 'yes' else 'no'
        
        return {
            'sport_norm': 'ANY',  # Generic templates match any sport
            'event_norm': 'ANY',
            'team_flag': team_flag,
            'result_type': result_type,
            'round_norm': round_norm,
            'medal_awarded': medal_awarded,
            'advancement': advancement
        }
    
    def map_template_raw_to_type(self) -> pd.DataFrame:
        """Create mapping from template_raw to template_type"""
        mappings = []
        
        print(f"Processing {len(self.template_raw)} template_raw rows...")
        print(f"Available template_type rows: {len(self.template_type)}")
        
        # Generate signatures for template_raw
        raw_signatures = []
        for idx, row in self.template_raw.iterrows():
            try:
                sig = self.derive_template_signature(row)
                sig['raw_idx'] = idx
                raw_signatures.append(sig)
            except Exception as e:
                print(f"Error processing template_raw row {idx}: {e}")
                continue
        
        print(f"Generated {len(raw_signatures)} raw signatures")
        
        # Generate signatures for template_type
        type_signatures = []
        for idx, row in self.template_type.iterrows():
            try:
                sig = self.derive_template_type_signature(row)
                sig['type_idx'] = idx
                sig['type_id'] = row.get('id', f'TYPE_{idx+1:03d}')
                type_signatures.append(sig)
            except Exception as e:
                print(f"Error processing template_type row {idx}: {e}")
                continue
        
        print(f"Generated {len(type_signatures)} type signatures")
        
        # Match raw to type
        for raw_sig in raw_signatures:
            best_match = None
            best_score = 0
            
            for type_sig in type_signatures:
                score = 0
                
                # Exact matches get higher scores
                if (type_sig['team_flag'] == raw_sig['team_flag']):
                    score += 2
                    
                if (type_sig['result_type'] == raw_sig['result_type']):
                    score += 2
                    
                if (type_sig['round_norm'] == raw_sig['round_norm'] or 
                    type_sig['round_norm'] == 'ANY' or
                    raw_sig['round_norm'] in type_sig['round_norm']):
                    score += 3  # Round is more important
                    
                if (type_sig['medal_awarded'] == raw_sig['medal_awarded'] or 
                    type_sig['medal_awarded'] == 'na'):
                    score += 1
                    
                if (type_sig['advancement'] == raw_sig['advancement'] or 
                    type_sig['advancement'] == 'na'):
                    score += 1
                
                if score > best_score:
                    best_score = score
                    best_match = type_sig
            
            if best_match:
                mappings.append({
                    'raw_idx': raw_sig['raw_idx'],
                    'type_idx': best_match['type_idx'],
                    'type_id': best_match['type_id'],
                    'sport_norm': raw_sig['sport_norm'],
                    'event_norm': raw_sig['event_norm'],
                    'team_flag': raw_sig['team_flag'],
                    'result_type': raw_sig['result_type'],
                    'round_norm': raw_sig['round_norm'],
                    'medal_awarded': raw_sig['medal_awarded'],
                    'advancement': raw_sig['advancement'],
                    'match_score': best_score
                })
        
        print(f"Created {len(mappings)} mappings")
        return pd.DataFrame(mappings)
    
    def pick_template_type(self, sport: str, event_name: str, round: str, 
                          is_team: bool = None, fields: List[str] = None) -> Optional[str]:
        """Pick the best template_type for given parameters"""
        
        # Normalize inputs
        sport_norm = self.sport_map.get(sport, sport.upper())
        event_norm = event_name.upper().strip() if event_name else ''
        round_norm = self.round_map.get(round, round.upper().strip())
        
        # Infer team flag if not provided
        if is_team is None:
            team_flag = self._infer_team_flag(event_name, '', fields or [])
        else:
            team_flag = 'yes' if is_team else 'no'
        
        # Get result type
        result_type = self._get_result_type(sport_norm, event_norm)
        
        # Find best match from template_type
        best_match = None
        best_score = 0
        
        for idx, row in self.template_type.iterrows():
            type_sig = self.derive_template_type_signature(row)
            score = 0
            
            # Calculate match score
            if type_sig['team_flag'] == team_flag:
                score += 2
            if type_sig['result_type'] == result_type:
                score += 2
            if type_sig['round_norm'] == round_norm or type_sig['round_norm'] == 'ANY':
                score += 3  # Round is most important
                
            if score > best_score:
                best_score = score
                best_match = row.get('id', f'TYPE_{idx+1:03d}')
        
        return best_match

# Usage example
def main():
    # Use the correct file path
    mapper = SportsTemplateMapper('data/all_sports.xlsx')
    
    # Create the mapping
    mapping_df = mapper.map_template_raw_to_type()
    print("Template Raw to Type Mapping:")
    print(f"Shape: {mapping_df.shape}")
    print(mapping_df.head(10))
    
    if not mapping_df.empty:
        # Show some statistics
        print(f"\nMapping Statistics:")
        print(f"Total mappings: {len(mapping_df)}")
        print(f"Unique template types used: {mapping_df['type_id'].nunique()}")
        print(f"Average match score: {mapping_df['match_score'].mean():.2f}")
        
        # Show distribution of template types
        print(f"\nTemplate Type Distribution:")
        print(mapping_df['type_id'].value_counts().head(10))
        
        # Show sport distribution
        print(f"\nSport Distribution:")
        print(mapping_df['sport_norm'].value_counts().head(10))
        
        # Example: Pick template for a specific case
        template_id = mapper.pick_template_type(
            sport='SWIMMING',
            event_name='200M Individual Medley',
            round='Final',
            is_team=False
        )
        print(f"\nRecommended template for Swimming 200M IM Final: {template_id}")
        
        # Save mapping to CSV
        mapping_df.to_csv('template_mapping.csv', index=False)
        print("\nMapping saved to template_mapping.csv")
    else:
        print("No mappings created. Check the data and logic.")

if __name__ == "__main__":
    main()


Processing 198 template_raw rows...
Available template_type rows: 28
Generated 198 raw signatures
Generated 28 type signatures
Created 198 mappings
Template Raw to Type Mapping:
Shape: (198, 11)
   raw_idx  type_idx  type_id    sport_norm           event_norm team_flag  \
0        0        12       13    GYMNASTICS  ARTISTIC GYMNASTICS        no   
1        1        27       28    GYMNASTICS  ARTISTIC GYMNASTICS        no   
2        2         1        2    GYMNASTICS  ARTISTIC GYMNASTICS       yes   
3        3         3        4  SEPAK TAKRAW              DOUBLES       yes   
4        4        18       19  SEPAK TAKRAW              DOUBLES       yes   
5        5        13       14  SEPAK TAKRAW              DOUBLES       yes   
6        6        10       11  SEPAK TAKRAW              DOUBLES       yes   
7        7        12       13  SEPAK TAKRAW             QUADRANT        no   
8        8         3        4  SEPAK TAKRAW                 REGU       yes   
9        9        27     

In [ ]:
import pandas as pd
import numpy as np

def classify_team_sports_and_create_mappings():
    """
    Fill out team sport classification and create template mappings for different round types
    """
    
    # Read the Excel file
    file_path = 'data/all_sports.xlsx'
    
    # Read the sport_result_mapping sheet
    sport_mapping_df = pd.read_excel(file_path, sheet_name='sport_result_mapping')
    
    # Read template_type sheet for reference
    template_type_df = pd.read_excel(file_path, sheet_name='template_type')
    
    # Define team sports based on the data
    team_sports = {
        'AQUATHLON': {'RELAY': 'yes'},
        'DUATHLON': {'RELAY': 'yes'},
        'TRIATHLON': {'RELAY': 'yes', 'MIXED RELAY': 'yes'},
        'ARTISTIC SWIMMING': {'DUET': 'yes', 'TEAM': 'yes', 'FREE COMBINATION': 'yes', 'HIGHLIGHT ROUTINE': 'yes'},
        'ATHLETICS': {'4X100M RELAY': 'yes', '4X400M RELAY': 'yes', 'MIXED 4X400M RELAY': 'yes'},
        'BADMINTON': {'MEN\'S DOUBLES': 'yes', 'WOMEN\'S DOUBLES': 'yes', 'TEAM': 'yes', 'MIXED DOUBLES': 'yes'},
        'BASEBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'BASKETBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes', '5X5': 'yes', '3X3': 'yes'},
        'BOWLING': {'DOUBLES': 'yes', 'TEAM OF 5': 'yes', 'ALL EVENTS': 'yes'},
        'CANOEING': {'C2': 'yes', 'K2': 'yes'},
        'CHESS': {'TEAM EVENTS': 'yes'},
        'CRICKET': {'T20': 'yes', 'ONE-DAY INTERNATIONAL (ODI)': 'yes', 'TEST MATCHES': 'yes'},
        'DIVING': {'SYNCHRONIZED DIVING': 'yes'},
        'DRAGON BOAT': {'200M': 'yes', '500M': 'yes', '1000M RACES': 'yes'},
        'ESPORTS': {'DOTA 2': 'yes', 'LEAGUE OF LEGENDS': 'yes', 'COUNTER-STRIKE': 'yes', 'VALORANT': 'yes', 'MOBILE LEGEND BANG BANG': 'yes', 'OVERWATCH': 'yes'},
        'FENCING': {'TEAM FOIL': 'yes', 'TEAM EPEE': 'yes', 'TEAM SABRE': 'yes'},
        'FLOORBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'FLYING DISC': {'ULTIMATE': 'yes'},
        'FOOTBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'GOLF': {'TEAM': 'yes'},
        'HANDBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'HOCKEY': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'ICE HOCKEY': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'JUDO': {'TEAM EVENTS': 'yes'},
        'KABADDI': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'KARATE': {'TEAM KATA': 'yes'},
        'NETBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'PENCAK SILAT': {'GANDA': 'yes', 'REGU': 'yes'},
        'PETANQUE': {'DOUBLES': 'yes', 'TRIPLES': 'yes'},
        'POLO': {'FIELD POLO': 'yes', 'ARENA POLO': 'yes'},
        'ROLLER SPORTS': {'INLINE HOCKEY': 'yes'},
        'ROWING': {'DOUBLE SCULLS': 'yes', 'QUADRUPLE SCULLS': 'yes', 'EIGHTS': 'yes'},
        'RUGBY': {'RUGBY SEVENS': 'yes', 'RUGBY UNION': 'yes'},
        'SEPAKTAKRAW': {'REGU': 'yes', 'DOUBLES REGU': 'yes'},
        'SOFTBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'SQUASH': {'DOUBLES': 'yes'},
        'SWIMMING': {'4X100M FREESTYLE RELAY': 'yes', '4X200M FREESTYLE RELAY': 'yes', '4X100M MEDLEY RELAY': 'yes', 'MIXED 4X100M FREESTYLE RELAY': 'yes', 'MIXED 4X100M MEDLEY RELAY': 'yes'},
        'TABLE TENNIS': {'DOUBLES': 'yes', 'MIXED DOUBLES': 'yes', 'TEAM EVENTS': 'yes'},
        'TENNIS': {'DOUBLES': 'yes', 'MIXED DOUBLES': 'yes'},
        'TEQBALL': {'DOUBLES': 'yes'},
        'TUG OF WAR': {'INDOOR': 'yes', 'OUTDOOR': 'yes'},
        'VOLLEYBALL': {'INDOOR VOLLEYBALL': 'yes', 'BEACH VOLLEYBALL': 'yes'},
        'WATER POLO': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'FLAG FOOTBALL': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'LACROSSE': {'MEN\'S TOURNAMENT': 'yes', 'WOMEN\'S TOURNAMENT': 'yes'},
        'WUSHU': {'DUILIAN': 'yes'},
        'ARCHERY': {'MIXED TEAM': 'yes'}
    }
    
    # Fill in the team column
    for index, row in sport_mapping_df.iterrows():
        sport = row['sport']
        value = row['value']
        
        if sport in team_sports and value in team_sports[sport]:
            sport_mapping_df.at[index, 'team'] = team_sports[sport][value]
        else:
            sport_mapping_df.at[index, 'team'] = 'no'
    
    # Define round types and their template mappings
    round_types = [
        'preliminaries', 'preliminary round', 'prelims',
        'qualification', 'qualifying',
        'heats', 'heat',
        'pool', 'pool stage',
        'round robin',
        'group stage', 'group',
        'round of 64', 'round of 32', 'round of 16',
        'table of 32', 'table of 16', 'table of 8',
        'quarter finals', 'quarter final',
        'semi finals', 'semi final',
        'bronze medal playoff', '3rd/4th placing match', '3rd placing match',
        'finals', 'final',
        'grand final'
    ]
    
    # Create template mappings for each sport-result type combination and round type
    template_mappings = []
    
    for _, sport_row in sport_mapping_df.iterrows():
        sport = sport_row['sport']
        value = sport_row['value']
        result_type = sport_row['resultType']
        is_team = sport_row['team']
        
        for round_type in round_types:
            # Determine appropriate template based on round type, result type, and team status
            template_id = get_template_id(round_type, result_type, is_team)
            
            mapping_row = {
                'sport': sport,
                'event': value,
                'round_type': round_type,
                'result_type': result_type,
                'is_team': is_team,
                'template_id': template_id
            }
            template_mappings.append(mapping_row)
    
    # Create DataFrame for template mappings
    template_mappings_df = pd.DataFrame(template_mappings)
    
    # Save the updated sport_result_mapping and new template mappings to Excel
    with pd.ExcelWriter('updated_all_sports.xlsx', engine='openpyxl') as writer:
        # Save updated sport_result_mapping
        sport_mapping_df.to_excel(writer, sheet_name='sport_result_mapping', index=False)
        
        # Save new template mappings
        template_mappings_df.to_excel(writer, sheet_name='round_template_mappings', index=False)
        
        # Copy other sheets from original file
        original_sheets = ['major_games_future_feature', 'pivot', 'template_raw', 'template_type', 'Misc', 'all_sports_raw', 'all_event_type', 'country_code']
        
        for sheet_name in original_sheets:
            try:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Could not copy sheet {sheet_name}: {e}")
    
    print("✅ Updated Excel file created: updated_all_sports.xlsx")
    print(f"✅ Added team sport classifications for {len(sport_mapping_df)} sport-event combinations")
    print(f"✅ Created {len(template_mappings_df)} template mappings for different round types")
    
    return sport_mapping_df, template_mappings_df

def get_template_id(round_type, result_type, is_team):
    """
    Determine the appropriate template ID based on round type, result type, and team status
    """
    round_lower = round_type.lower()
    
    # Finals templates
    if 'final' in round_lower:
        if is_team == 'yes':
            if result_type == 'score':
                return 1  # Team score final
            elif result_type == 'time':
                return 2  # Team time final
            elif result_type == 'combat':
                return 1  # Team combat final
        else:
            if result_type == 'score':
                return 1  # Individual score final
            elif result_type == 'time':
                return 5  # Individual time final
            elif result_type == 'height':
                return 9  # Individual height final
            elif result_type == 'distance':
                return 1  # Individual distance final
            elif result_type == 'weight':
                return 1  # Individual weight final
            elif result_type == 'placing':
                return 18  # Individual placing final
            elif result_type == 'combat':
                return 1  # Individual combat final
    
    # Preliminary/qualifying rounds
    elif any(x in round_lower for x in ['prelim', 'qualif', 'heat']):
        if result_type == 'time':
            return 6  # Time heat/qualifying
        elif is_team == 'yes':
            return 16  # Team preliminary
        else:
            return 3  # Individual qualifying
    
    # Pool rounds
    elif 'pool' in round_lower:
        if result_type == 'bouts':
            return 21  # Pool with bouts
        elif is_team == 'yes':
            return 4  # Team pool
        else:
            return 3  # Individual pool
    
    # Round robin
    elif 'round robin' in round_lower:
        if is_team == 'yes':
            return 4  # Team round robin
        else:
            return 4  # Individual round robin
    
    # Group stage
    elif 'group' in round_lower:
        if is_team == 'yes':
            return 4  # Team group stage
        else:
            return 3  # Individual group stage
    
    # Knockout rounds
    elif any(x in round_lower for x in ['quarter', 'semi', 'round of']):
        if is_team == 'yes':
            if 'quarter' in round_lower:
                return 11  # Team quarter finals
            elif 'semi' in round_lower:
                return 14  # Team semi finals
            else:
                return 11  # Team knockout
        else:
            if 'round of 32' in round_lower:
                return 12  # Individual round of 32
            else:
                return 13  # Individual knockout
    
    # Placing matches
    elif any(x in round_lower for x in ['placing', 'playoff']):
        if 'bronze' in round_lower or '3rd' in round_lower:
            return 17  # Bronze medal playoff
        else:
            return 15  # Other placing matches
    
    # Default
    return 1

# Run the function
if __name__ == "__main__":
    sport_df, template_df = classify_team_sports_and_create_mappings()
    
    # Display some sample results
    print("\n📊 Sample team sport classifications:")
    team_sports_sample = sport_df[sport_df['team'] == 'yes'].head(10)
    print(team_sports_sample[['sport', 'value', 'team']])
    
    print("\n📋 Sample template mappings:")
    template_sample = template_df.head(10)
    print(template_sample[['sport', 'event', 'round_type', 'template_id']])
    
    print(f"\n📈 Summary:")
    print(f"Total team events: {len(sport_df[sport_df['team'] == 'yes'])}")
    print(f"Total individual events: {len(sport_df[sport_df['team'] == 'no'])}")
    print(f"Total template mappings created: {len(template_df)}")


FileNotFoundError: [Errno 2] No such file or directory: 'all_sports.xlsx'